In [2]:
import pandas as pd
from tqdm import tqdm     

In [3]:
df = pd.read_csv("train.csv")
df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12178 entries, 0 to 12177
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Case Name       12178 non-null  object
 1   Input           12178 non-null  object
 2   Output          12178 non-null  object
 3   Label           12178 non-null  int64 
 4   Count           12178 non-null  int64 
 5   Decision_Count  12178 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 571.0+ KB


In [4]:
def preprocess_case(text):
  max_tokens = 1000
  tokens = text.split(' ')
  num_tokens_to_extract = min(max_tokens, len(tokens))
  text1 = ' '.join(tokens[-num_tokens_to_extract:len(tokens)])
  return text1
     

for i,row in df.iterrows():
  input = row['Input']
  input = preprocess_case(input)
  df.at[i,'Input'] = input

In [5]:
def format_instruction(sample, inst):
	return f"""### Instruction:
{inst}

### Input:
{sample['Input']}

### Response:
{sample['Output'].split('[ds]')[0]}

### Explanation:
{preprocess_case(sample['Output'].split('[ds]')[1])}
"""
#prediction

# def format_instruction(sample, inst):
# 	return f"""### Instruction:
# {inst}

# ### Input:
# {sample['Input']}

# ### Response:
# {sample['Output'].split('[ds]')[0]}
# """

In [6]:
df_ins = pd.read_csv("instruction_decision.csv")
     

import random
random.seed(15)
text = []
for i,row in tqdm(df.iterrows()):
  random_index = random.randint(0, len(df_ins) - 1)
  t = format_instruction(df.iloc[i],df_ins['Instructions_Exp'][random_index])
  #t = format_instruction(df.iloc[i],df_ins['Instructions'][random_index])
  text.append(t)

12178it [00:03, 3898.90it/s]


In [7]:
df.loc[:, "text"] = text
     

# Set a random state for reproducibility
random_state = 42

# Calculate the number of rows for validation set (10%)
val_size = int(0.1 * len(df))

df_val = df.sample(n=val_size, random_state=random_state)

df_train = df.drop(df_val.index)

df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
     

df_train.to_csv("train_ft.csv",index=False)
df_val.to_csv("val_ft.csv",index=False)